In [115]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, ReLU
from keras import optimizers, layers
from tensorflow import math
import warnings
warnings.filterwarnings('ignore')

In [35]:
rain = pd.read_csv("./Data/Rainfall.csv")

In [36]:
import glob

slp_path = "./Data/slp/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)
  
slp_months.sort()
print(slp_months)

['./Data/slp\\01_slp.csv', './Data/slp\\02_slp.csv', './Data/slp\\03_slp.csv', './Data/slp\\04_slp.csv', './Data/slp\\05_slp.csv', './Data/slp\\06_slp.csv', './Data/slp\\07_slp.csv', './Data/slp\\08_slp.csv', './Data/slp\\09_slp.csv', './Data/slp\\10_slp.csv', './Data/slp\\11_slp.csv', './Data/slp\\12_slp.csv']


In [37]:
#Here i am just storing rainfall data of Jun to sep from 1949 to 2014
# index 47= 1948,
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:89]))
print(lpa)
rain_fall_data = list(rain['Jun-Sep'][47:2020])
rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

878.0689655172414


In [38]:
from scipy.stats import pearsonr
correlation = []

# chacking for each MONTH
month_path = slp_months[1] #0th in dex means january
# reading month data
month_data = pd.read_csv(month_path)
# month_data.head()
# len(month_data)

In [39]:
month_data.iloc[0]
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

combine = []
for i in range(12):
  month_path = slp_months[i]
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:53]
  for i in feature:
    combine.append(i)
x = np.array(combine)
x.shape

(636, 324)

In [40]:
def get_lr(initial_learning_rate=0.001, decay_steps=4000, decay_rate=1):
    return optimizers.schedules.ExponentialDecay(
            initial_learning_rate,
            decay_steps=decay_steps,
            decay_rate=decay_rate,
            staircase=True)

In [41]:
def get_model_1(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  bottleneck = Dense(97)(e)
  output = Dense(n_inputs, activation='linear')(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [42]:
model_1 = get_model_1()
model_1.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_16 (Dense)            (None, 324)               105300    
                                                                 
 re_lu_8 (ReLU)              (None, 324)               0         
                                                                 
 dense_17 (Dense)            (None, 97)                31525     
                                                                 
 dense_18 (Dense)            (None, 324)               31752     
                                                                 
Total params: 168,577
Trainable params: 168,577
Non-trainable params: 0
_________________________________________________________________


In [43]:
history = model_1.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/800
40/40 - 0s - loss: 10739.8887 - val_loss: 6528.5200 - 332ms/epoch - 8ms/step
Epoch 2/800
40/40 - 0s - loss: 5501.0923 - val_loss: 4288.5957 - 67ms/epoch - 2ms/step
Epoch 3/800
40/40 - 0s - loss: 3772.5708 - val_loss: 3128.8735 - 68ms/epoch - 2ms/step
Epoch 4/800
40/40 - 0s - loss: 2878.0376 - val_loss: 2437.9136 - 68ms/epoch - 2ms/step
Epoch 5/800
40/40 - 0s - loss: 2318.2039 - val_loss: 2002.7893 - 65ms/epoch - 2ms/step
Epoch 6/800
40/40 - 0s - loss: 1940.6073 - val_loss: 1707.8118 - 67ms/epoch - 2ms/step
Epoch 7/800
40/40 - 0s - loss: 1675.2993 - val_loss: 1496.9850 - 67ms/epoch - 2ms/step
Epoch 8/800
40/40 - 0s - loss: 1478.3582 - val_loss: 1320.5139 - 66ms/epoch - 2ms/step
Epoch 9/800
40/40 - 0s - loss: 1333.4806 - val_loss: 1200.7875 - 66ms/epoch - 2ms/step
Epoch 10/800
40/40 - 0s - loss: 1215.5414 - val_loss: 1107.2628 - 67ms/epoch - 2ms/step
Epoch 11/800
40/40 - 0s - loss: 1120.8270 - val_loss: 1030.9891 - 68ms/epoch - 2ms/step
Epoch 12/800
40/40 - 0s - loss: 1039.84

In [44]:
model_1.layers[3].get_weights()[0].shape

(324, 97)

In [45]:
model_temp1 = Model(inputs=model_1.input, outputs=model_1.layers[3].output)
model_2_inputs = model_temp1(x)
model_2_inputs.shape

TensorShape([636, 97])

In [46]:
# define encoder
def get_model_2(n_inputs=97):
  # visible = Input(shape=(n_inputs,))

  # # e = Dense(n_inputs, use_bias=False)(visible)
  # e = ReLU()(visible)
  # bottleneck = Dense(29, use_bias=False)(e)
  # output = Dense(n_inputs)(bottleneck)

  # model = Model(inputs=visible, outputs=output)
  # model.compile(optimizer='adam', loss='mse')
  # return model

  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  bottleneck = Dense(29)(e)
  output = Dense(n_inputs, activation='linear')(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [47]:
model_2 = get_model_2()
model_2.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 97)]              0         
                                                                 
 dense_19 (Dense)            (None, 97)                9506      
                                                                 
 re_lu_9 (ReLU)              (None, 97)                0         
                                                                 
 dense_20 (Dense)            (None, 29)                2842      
                                                                 
 dense_21 (Dense)            (None, 97)                2910      
                                                                 
Total params: 15,258
Trainable params: 15,258
Non-trainable params: 0
_________________________________________________________________


In [48]:
history = model_2.fit(model_2_inputs, model_2_inputs, epochs=800, batch_size=16, verbose=2, validation_data=(model_2_inputs,model_2_inputs))

Epoch 1/800
40/40 - 0s - loss: 24437.9316 - val_loss: 20349.1738 - 415ms/epoch - 10ms/step
Epoch 2/800
40/40 - 0s - loss: 18451.2285 - val_loss: 16249.4746 - 79ms/epoch - 2ms/step
Epoch 3/800
40/40 - 0s - loss: 14885.4844 - val_loss: 13547.2686 - 73ms/epoch - 2ms/step
Epoch 4/800
40/40 - 0s - loss: 12907.1787 - val_loss: 12052.7500 - 65ms/epoch - 2ms/step
Epoch 5/800
40/40 - 0s - loss: 11672.0967 - val_loss: 11003.3535 - 63ms/epoch - 2ms/step
Epoch 6/800
40/40 - 0s - loss: 10705.9170 - val_loss: 10143.1465 - 69ms/epoch - 2ms/step
Epoch 7/800
40/40 - 0s - loss: 9912.3916 - val_loss: 9432.5586 - 67ms/epoch - 2ms/step
Epoch 8/800
40/40 - 0s - loss: 9264.3389 - val_loss: 8845.6270 - 75ms/epoch - 2ms/step
Epoch 9/800
40/40 - 0s - loss: 8731.3789 - val_loss: 8362.1367 - 74ms/epoch - 2ms/step
Epoch 10/800
40/40 - 0s - loss: 8285.9951 - val_loss: 7965.5762 - 66ms/epoch - 2ms/step
Epoch 11/800
40/40 - 0s - loss: 7923.9126 - val_loss: 7631.4766 - 81ms/epoch - 2ms/step
Epoch 12/800
40/40 - 0s - l

In [49]:
model_temp2 = Model(inputs=model_2.input, outputs=model_2.layers[3].output)
model_3_inputs = model_temp2(model_2_inputs)
model_3_inputs.shape

TensorShape([636, 29])

In [73]:
def get_model_3(n_inputs=29):
  # visible = Input(shape=(n_inputs,))

  # # e = Dense(n_inputs, use_bias=False)(visible)
  # e = ReLU()(visible)
  # bottleneck = Dense(9)(e)
  # output = Dense(n_inputs, use_bias=False)(bottleneck)

  # model = Model(inputs=visible, outputs=output)
  # model.compile(optimizer='adam', loss='mse')
  # return model

  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  bottleneck = Dense(9)(e)
  output = Dense(n_inputs, activation='linear')(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
  return model

In [80]:
model_3 = get_model_3()
model_3.summary()

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 29)]              0         
                                                                 
 dense_44 (Dense)            (None, 29)                870       
                                                                 
 re_lu_20 (ReLU)             (None, 29)                0         
                                                                 
 dense_45 (Dense)            (None, 9)                 270       
                                                                 
 dense_46 (Dense)            (None, 29)                290       
                                                                 
Total params: 1,430
Trainable params: 1,430
Non-trainable params: 0
_________________________________________________________________


In [81]:
# fit the autoencoder model to reconstruct input
history = model_3.fit(model_3_inputs, model_3_inputs, epochs=400, batch_size=16, verbose=2, validation_data=(model_3_inputs,model_3_inputs))
# plot loss

Epoch 1/400
20/20 - 0s - loss: 19810.2227 - val_loss: 18396.1602 - 318ms/epoch - 16ms/step
Epoch 2/400
20/20 - 0s - loss: 17586.2930 - val_loss: 16778.5449 - 31ms/epoch - 2ms/step
Epoch 3/400
20/20 - 0s - loss: 16256.4199 - val_loss: 15665.8936 - 35ms/epoch - 2ms/step
Epoch 4/400
20/20 - 0s - loss: 15290.8867 - val_loss: 14846.2246 - 36ms/epoch - 2ms/step
Epoch 5/400
20/20 - 0s - loss: 14543.9170 - val_loss: 14184.8379 - 34ms/epoch - 2ms/step
Epoch 6/400
20/20 - 0s - loss: 13924.1162 - val_loss: 13614.1670 - 53ms/epoch - 3ms/step
Epoch 7/400
20/20 - 0s - loss: 13381.9336 - val_loss: 13107.5830 - 35ms/epoch - 2ms/step
Epoch 8/400
20/20 - 0s - loss: 12901.1592 - val_loss: 12652.1055 - 34ms/epoch - 2ms/step
Epoch 9/400
20/20 - 0s - loss: 12468.7314 - val_loss: 12244.3613 - 34ms/epoch - 2ms/step
Epoch 10/400
20/20 - 0s - loss: 12083.7959 - val_loss: 11880.7959 - 37ms/epoch - 2ms/step
Epoch 11/400
20/20 - 0s - loss: 11744.7686 - val_loss: 11565.7559 - 39ms/epoch - 2ms/step
Epoch 12/400
20/2

In [53]:
def get_model_final(n_inputs=324):
  # visible = Input(shape=(n_inputs,))
  # # e = Dense(n_inputs, use_bias=False)(visible)
  # e = ReLU()(visible)
  # e = Dense(97, use_bias=False)(e)
  # e = ReLU()(e)
  # e = Dense(29, use_bias=False)(e)
  # e = ReLU()(e)
  # bottleneck = Dense(9)(e)
  # e = Dense(29)(bottleneck)
  # e = ReLU()(e)
  # e = Dense(97)(e)
  # e = ReLU()(e)
  # output = Dense(n_inputs, activation='linear')(e)

  # model = Model(inputs=visible, outputs=output)
  # model.compile(optimizer='adam', loss='mse')
  # return model

  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  e = Dense(97)(e)
  e = ReLU()(e)
  e = Dense(29)(e)
  e = ReLU()(e)
  bottleneck = Dense(9)(e)
  e = Dense(29)(bottleneck)
  e = ReLU()(e)
  e = Dense(97)(e)
  e = ReLU()(e)
  output = Dense(n_inputs, activation='linear')(e)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [82]:
model_final = get_model_final()
model_final.summary()

Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 324)]             0         
                                                                 
 dense_47 (Dense)            (None, 324)               105300    
                                                                 
 re_lu_21 (ReLU)             (None, 324)               0         
                                                                 
 dense_48 (Dense)            (None, 97)                31525     
                                                                 
 re_lu_22 (ReLU)             (None, 97)                0         
                                                                 
 dense_49 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_23 (ReLU)             (None, 29)                0  

In [83]:
model_final.layers[2] = model_1.layers[3]
model_final.layers[11] = model_1.layers[4]
model_final.layers[4] = model_2.layers[3]
model_final.layers[9] = model_2.layers[4]
model_final.layers[6] = model_3.layers[3]
model_final.layers[7] = model_3.layers[4]
model_final.compile(optimizer='adam', loss='mse')

In [84]:
model_final.summary()

Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 324)]             0         
                                                                 
 dense_47 (Dense)            (None, 324)               105300    
                                                                 
 re_lu_21 (ReLU)             (None, 324)               0         
                                                                 
 dense_48 (Dense)            (None, 97)                31525     
                                                                 
 re_lu_22 (ReLU)             (None, 97)                0         
                                                                 
 dense_49 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_23 (ReLU)             (None, 29)                0  

In [85]:
history = model_final.fit(x, x, epochs=1000, batch_size=16, verbose=2, validation_data=(x,x))


Epoch 1/1000
40/40 - 0s - loss: 13173.3799 - val_loss: 10054.8740 - 481ms/epoch - 12ms/step
Epoch 2/1000
40/40 - 0s - loss: 8829.2725 - val_loss: 7952.5957 - 74ms/epoch - 2ms/step
Epoch 3/1000
40/40 - 0s - loss: 7868.5771 - val_loss: 7648.0737 - 99ms/epoch - 2ms/step
Epoch 4/1000
40/40 - 0s - loss: 7573.6118 - val_loss: 7287.8325 - 71ms/epoch - 2ms/step
Epoch 5/1000
40/40 - 0s - loss: 7099.3218 - val_loss: 6707.3247 - 87ms/epoch - 2ms/step
Epoch 6/1000
40/40 - 0s - loss: 6568.4263 - val_loss: 6258.3726 - 92ms/epoch - 2ms/step
Epoch 7/1000
40/40 - 0s - loss: 6174.1445 - val_loss: 5929.7271 - 84ms/epoch - 2ms/step
Epoch 8/1000
40/40 - 0s - loss: 5880.1689 - val_loss: 5659.7290 - 75ms/epoch - 2ms/step
Epoch 9/1000
40/40 - 0s - loss: 5632.3457 - val_loss: 5394.7515 - 107ms/epoch - 3ms/step
Epoch 10/1000
40/40 - 0s - loss: 5418.0527 - val_loss: 5225.8184 - 81ms/epoch - 2ms/step
Epoch 11/1000
40/40 - 0s - loss: 5200.8369 - val_loss: 5079.5288 - 104ms/epoch - 3ms/step
Epoch 12/1000
40/40 - 0s

In [106]:
# model_final.save("./Data/Models/model_final.h5")
model_final = keras.models.load_model('./Data/Models/model_final.h5')

In [101]:
class Custom_Layer(layers.Layer):
    def __init__(self, weights, bias=False):
        super(Custom_Layer, self).__init__()
        self.w = weights
        if bias:
            self.b = bias

    def call(self, inputs):
        try: return tf.matmul(inputs, self.w) + self.b
        except: return tf.matmul(inputs, self.w)

In [107]:
std = []
for i in model_final.layers:
    # print(i.weights[0])
    try: std.append(np.array(math.reduce_std(i.weights[0])) * 2)
    except: std.append([])

In [108]:
final_layers = keras.Sequential([Input(shape=(None, 324))])
for i, layer in enumerate(model_final.layers):
    try:
        temp_layer = []
        for ind, j in enumerate(layer.weights[0]):
            temp_weights = []
            for index, k in enumerate(j):
                if k < std[i]: temp_weights.append(0)
                else:          temp_weights.append(1)  
            temp_layer.append(temp_weights)
        weights = tf.Variable(temp_layer, dtype='float32')
        try: final_layers.add(Custom_Layer(weights, bias=layer.weights[1]))
        except: final_layers.add(Custom_Layer(weights))
    except IndexError: final_layers.add(layer)

In [109]:
final_layers.compile(optimizer='adam', loss='mse')
final_layers.build(input_shape=(None, 324))
final_layers.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       multiple                  0         
                                                                 
 custom__layer_83 (Custom_La  (None, None, 324)        104976    
 yer)                                                            
                                                                 
 re_lu_30 (ReLU)             multiple                  0         
                                                                 
 custom__layer_85 (Custom_La  (None, None, 97)         31428     
 yer)                                                            
                                                                 
 re_lu_31 (ReLU)             multiple                  0         
                                                                 
 custom__layer_87 (Custom_La  (None, None, 29)        

In [110]:
model_final1 = Model(inputs=model_final.input, outputs=model_final.layers[3].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_final.input, outputs=model_final.layers[5].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_final.input, outputs=model_final.layers[7].output)
model_final3.trainable = False

models = [model_final1, model_final2, model_final3]
# models = [model_final1, model_final4]

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 324), dtype=tf.float32, name='input_24'), name='input_24', description="created by layer 'input_24'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 324), dtype=tf.float32, name='input_24'), name='input_24', description="created by layer 'input_24'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 324), dtype=tf.float32, name='input_24'), name='input_24', description="created by layer 'input_24'")


In [111]:
final_layers1 = Model(inputs=final_layers.input, outputs=final_layers.layers[3].output)
final_layers1.trainable = False

final_layers2 = Model(inputs=final_layers.input, outputs=final_layers.layers[5].output)
final_layers2.trainable = False

final_layers3 = Model(inputs=final_layers.input, outputs=final_layers.layers[7].output)
final_layers3.trainable = False

models = [final_layers1, final_layers2, final_layers3]

In [112]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
def predictor(month,mod):
  # chacking for each MONTH
  month_path = slp_months[month] #0th in dex means january
  # reading month data
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:]  #Here i am using anly data from 1948 to 2000
  pred = np.array(feature)
  try: pred_m = models[mod](feature)
  except: 
    print(i)
    raise(IndexError)
  pred_f = pred_m.numpy()
  return pred_f

In [113]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_info,w_start,w_end):
  k = 0
  pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  for i in months:
    for mod in range(len(models)):
      y_pred = predictor(i,mod)
      b = get_top_pred(top, y_pred, w_start,w_end)
      try:    pred = np.concatenate((pred,b), axis=1)
      except: pred = np.array(b)
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      score.append(reg.predict([pred[53+i]])[0])
    # print(score)
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [116]:
from itertools import combinations
# months_comb = [j for i in range(4) for j in combinations(range(5,9), i+1)]
months_comb = [[4]]
top = 9
# models = [model_final1, model_final2, model_final3]
models = [final_layers1, final_layers2, final_layers3]

minimum, maximum = 1, -1
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print("\n\nMonth combination = ",month,"    top = ",i,"   min and max",left,"   ",right)
    if minimum > left:  minimum, min_month, min_top = left, month, i
    if maximum < right: maximum, max_month, max_top = right, month, i
    print(f'Minimum: {minimum}, {min_month}, {min_top}    Maximum: {maximum}, {max_month}, {max_top}\n')

Window size =  10    plcc -0.22358921934127696
Window size =  11    plcc -0.31147481798276644
Window size =  12    plcc -0.34107895223883034
Window size =  13    plcc -0.04910743886370269
Window size =  14    plcc -0.3985208051750466
Window size =  15    plcc -0.5488590167863505
Window size =  16    plcc -0.3676585007916507
Window size =  17    plcc -0.558547547105239
Window size =  18    plcc -0.4104257141327139
Window size =  19    plcc -0.3545197424582842
Window size =  20    plcc -0.38448205228010496
Window size =  21    plcc -0.02396376953834158
Window size =  22    plcc -0.36992136919860313
Window size =  23    plcc -0.24578859346274115
Window size =  24    plcc -0.3859226969315467
Window size =  25    plcc -0.416616352657228
Window size =  26    plcc -0.6972336455299376
Window size =  27    plcc -0.5170066369013365
Window size =  28    plcc -0.6740742332982341
Window size =  29    plcc -0.6133407266925329
Window size =  30    plcc -0.6572680614073751
Window size =  31    plcc -0